In [ ]:
%load_ext autoreload
%autoreload 2

%config InlineBackend.figure_format = 'retina'

In [ ]:
import scanpy as sc 
import numpy as np
import os
import pandas as pd

In [ ]:
import celloracle as co
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams["figure.figsize"] = (5, 5)
plt.rcParams['figure.dpi'] = 180


In [ ]:
adata = sc.read_h5ad('/ix/djishnu/shared/djishnu_kor11/training_data_2025/mLND3-1_v4.h5ad')
adata

In [ ]:
oracle = co.load_hdf5("/ix/djishnu/shared/djishnu_kor11/co_objects/mLND3-1_v4.celloracle.oracle")
links = co.load_hdf5(file_path="/ix/djishnu/shared/djishnu_kor11/co_objects/mLND3-1_v4-links.celloracle.links")

In [ ]:
tfs = []
for cluster, df in links.links_dict.items():
    tfs.extend(df["source"].unique())

tfs = sorted(set(tfs))
len(tfs)

In [ ]:
tf = 'Runx1'
oracle.simulate_shift(perturb_condition={tf: 0})

In [ ]:
import json
with open('../../data/GSEA/m2.all.v2024.1.Mm.json', 'r') as f:
    gsea_modules = json.load(f)

In [ ]:
import sys
sys.path.append('../../src')
from spaceoracle.plotting.gsea import * 

gsea_scores = compute_gsea_scores(oracle.adata, gsea_modules)
gsea_scores_perturbed = compute_gsea_scores(oracle.adata, gsea_modules, layer='simulated_count')

Computing GSEA scores:  80%|███████▉  | 2156/2710 [59:16<1:12:16,  7.83s/it]

In [ ]:
delta_gsea_scores = gsea_scores - gsea_scores_perturbed
delta_gsea_scores.dropna(inplace=True)

delta_gsea_scores['abs_mean'] = delta_gsea_scores.iloc[:, :-1].apply(lambda row: np.abs(row.mean()), axis=1)
delta_gsea_scores.sort_values(by = 'abs_mean', ascending=False, inplace=True)

delta_gsea_scores = delta_gsea_scores.loc[delta_gsea_scores.index.str.contains('BIOCARTA')]
delta_gsea_scores

In [ ]:
show_gsea_scores(
    oracle.adata, delta_gsea_scores, annot='cell_type', n_show=9, 
    savepath=f'/ix/djishnu/shared/djishnu_kor11/co_results/mLDN3-1_v4/{tf}/gsea.png')

In [ ]:
from spaceoracle.plotting.transitions import contour_shift
from spaceoracle.judges import Judge, permute_rows_nsign

judger = Judge(adata, annot='cell_type')


In [ ]:
results_dir = '/ix/djishnu/shared/djishnu_kor11/co_results/mLDN3-1_v4'
st_dir = '/ix/djishnu/shared/djishnu_kor11/perturbations/mLDN3-1_v4'

os.mkdir(f'{results_dir}/{tf}')

In [ ]:
seed=1334

fig, axs = plt.subplots(1, 2, figsize=(16, 8), gridspec_kw={'width_ratios': [1, 1]})
axs.flatten()
contour_shift(oracle.adata, title=f'Cell Identity Shift from {tf} KO', annot='cell_type', seed=seed, ax=axs[0])

delta_X_rndm = oracle.adata.layers['delta_X'].copy()
permute_rows_nsign(delta_X_rndm)
fake_simulated_count = oracle.adata.layers['imputed_count'] + delta_X_rndm

contour_shift(oracle.adata, title=f'Randomized Effect of {tf} KO Shift', annot='cell_type', seed=seed, ax=axs[1], perturbed=fake_simulated_count)

plt.tight_layout()
plt.savefig(f'{results_dir}/{tf}/contour.png')

In [ ]:
nt = pd.DataFrame(oracle.adata.layers['imputed_count'], columns=oracle.adata.var_names)
co = pd.DataFrame(oracle.adata.layers['simulated_count'], columns=oracle.adata.var_names)
st = pd.read_parquet(f'{st_dir}/{tf}.parquet')

In [ ]:
co_sim_adata = judger.create_sim_adata(nt.values, co.values)
co_deg_df = judger.get_expected_degs(co_sim_adata, tf, show=30, save_path=results_dir+f'/{tf}/degs_CO.png')

In [ ]:
st_sim_adata = judger.create_sim_adata(nt.values, st.values)
st_deg_df = judger.get_expected_degs(st_sim_adata, tf, show=30, save_path=results_dir+f'/{tf}/degs_ST.png')

In [ ]:
judger.plot_delta_corr(nt=nt, co=co, pred=st, ko=tf, save_path=results_dir+f'/{tf}/delta_corr.png')